In [ ]:
import hashlib
from pathlib import Path

import numpy as np
import pandas as pd
import tqdm

from act_rec.labeling import YoloPoseVideoLabeler
from act_rec.params import YoloPoseVideoInferenceParams


In [ ]:
BASE_DIR = Path.cwd().parent

In [ ]:
params = YoloPoseVideoInferenceParams()
labeler = YoloPoseVideoLabeler(
    model_path="../../models/yolo11x-pose.pt",
    params=params,
)

In [ ]:
labels = pd.read_csv(BASE_DIR / "labels.csv")

In [ ]:
labels.dropna()["label"]

In [ ]:
n_frames = []
skeleton_paths = []

for path in tqdm.tqdm(labels["video_path"]):
    absolute_video_path = BASE_DIR / path
    is_single_person, keypoints = labeler.label_video(
        absolute_video_path,
        multi_person_tollerance_n_frames=15,
    )
    if not is_single_person:
        n_frames.append(None)
        skeleton_paths.append(None)
        continue
    skeleton_file_path = Path("skeletons") / f"{hashlib.md5(path.encode()).hexdigest()}.npy"
    absolute_skeleton_file_path = BASE_DIR / skeleton_file_path
    absolute_skeleton_file_path.parent.mkdir(parents=True, exist_ok=True)
    np.save(absolute_skeleton_file_path, keypoints)
    n_frames.append(keypoints.shape[0])
    skeleton_paths.append(skeleton_file_path)

In [ ]:
labels["n_frames"] = n_frames
labels["skeleton_path"] = skeleton_paths

In [ ]:
labels.to_csv(BASE_DIR / "labels.csv", index=False)